In [38]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import os 

In [39]:
path = os.getcwd() + '/ml1m/'

In [40]:
# Load the data
ratings_df = pd.read_csv(f'{path}preprocessed/ratings.txt', sep='\t')
ratings_df


,uid,pid,rating,timestamp
0,0,872,5,978300760
1,0,537,3,978302109
2,0,679,3,978301968
3,0,2606,4,978300275
4,0,1790,5,978824291
...,...,...,...,...
940958,6039,810,1,956716541
940959,6039,813,5,956704887
940960,6039,477,5,956704746
940961,6039,815,4,956715648


In [41]:
movies_df = pd.read_csv(f'{path}movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')
movies_df

/var/folders/8j/vb8qq8v168531_7qk_cwrfyc0000gn/T/ipykernel_8848/736248507.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies_df = pd.read_csv(f'{path}movies.dat',  sep="::", names=["movie_id", "movie_name", "genre"], header=None, encoding='latin-1')


,movie_id,movie_name,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [42]:
mapping_df = pd.read_csv(f'{path}/preprocessed/products.txt', sep='\t', index_col='new_id')
mapping_df

,raw_dataset_id
new_id,
0,2
1,3
2,4
3,5
4,6
...,...
3025,3947
3026,3948
3027,3949


In [62]:
movies = pd.merge(movies_df, mapping_df, left_on='movie_id', right_on='raw_dataset_id')
# movies.set_index('Unnamed: 0',inplace=True)
movies['id'] = range(len(movies))
movies.set_index('id', inplace=True)
movies = movies.drop(columns=['movie_id', 'raw_dataset_id'])

In [63]:
movies

,movie_name,genre
id,,
0,Jumanji (1995),Adventure|Children's|Fantasy
1,Grumpier Old Men (1995),Comedy|Romance
2,Waiting to Exhale (1995),Comedy|Drama
3,Father of the Bride Part II (1995),Comedy
4,Heat (1995),Action|Crime|Thriller
...,...,...
3025,Get Carter (1971),Thriller
3026,Meet the Parents (2000),Comedy
3027,Requiem for a Dream (2000),Drama


In [67]:
data = pd.merge(ratings_df, movies, left_on='pid', right_on='id')
data

,uid,pid,rating,timestamp,movie_name,genre
0,0,872,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,872,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,11,872,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,14,872,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,16,872,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
940958,2908,2634,5,971971272,Born American (1986),Action|Drama|Thriller
940959,3313,2634,3,967947206,Born American (1986),Action|Drama|Thriller
940960,3617,2634,3,966598532,Born American (1986),Action|Drama|Thriller
940961,5678,2634,4,958619701,Born American (1986),Action|Drama|Thriller


In [65]:
# d = pd.merge(d, movies_df, left_on='raw_dataset_id', right_on='movie_id')
# d

In [68]:
# d = d
# df

In [69]:
# Create a sparse user-movie matrix
user_movie_matrix = data.pivot_table(index='uid', columns='movie_name', values='rating').fillna(0)
user_movie_matrix

movie_name,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
uid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,3.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,1.0,...,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,2.0
6036,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
genre_matrix = movies.genre.str.get_dummies().astype(bool)
genre_matrix

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
id,,,,,,,,,,,,,,,,,,
0,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False
2,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
3026,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3027,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False


In [74]:
user_movie_matrix_norm = user_movie_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1).clip(lower=0)
user_movie_matrix_norm
#make min

movie_name,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),"13th Warrior, The (1999)",...,Young Doctors in Love (1982),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zeus and Roxanne (1997),eXistenZ (1999)
uid,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.987195,0.787195,0.0,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.0,0.421452,0.0,0.0,0.0,0.221452,0.621452,0.0,0.000000,0.021452,...,0.0,0.621452,0.000000,0.000000,0.0,0.421452,0.0,0.0,0.0,0.221452
6036,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.753201,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
6037,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [75]:
# Perform NMF on the user-item matrix
nmf = NMF(n_components=30, init='nndsvd', max_iter=1000, random_state=0)
user_factors = nmf.fit_transform(user_movie_matrix_norm)
item_factors = nmf.components_

In [76]:
item_similarity = cosine_similarity(item_factors.T)

In [77]:
item_genre_matrix = csr_matrix(genre_matrix.values)

In [78]:
genre_similarity = cosine_similarity(item_genre_matrix)

In [79]:
# Set weights for hybrid recommender
alpha = 0.5
beta = 0.5

In [80]:
# Calculate weighted similarity matrix
weighted_similarity = alpha * item_similarity + beta * genre_similarity

In [81]:
# Make recommendations
def recommend_movies(user_id, user_item_matrix, item_similarity, genre_matrix, weighted_similarity, k=10):
    user_items = user_item_matrix.loc[user_id].to_numpy().reshape(1, -1)
    item_scores = np.dot(user_items, item_similarity).reshape(-1, 1)
    genre_scores = np.dot(item_similarity.T, np.dot(genre_matrix, user_items.T)).reshape(-1, 1)
    weighted_scores = np.dot(weighted_similarity, user_items.T).reshape(-1, 1)
    scores = alpha * item_scores + beta * genre_scores + (1 - alpha - beta) * weighted_scores
    scores_df = pd.DataFrame(scores, index=user_item_matrix.columns, columns=['score'])
    scores_df = scores_df.loc[~user_item_matrix.loc[user_id].notna()]
    scores_df = scores_df.sort_values(by='score', ascending=False).head(k)
    return list(scores_df.index)

In [82]:

# Test the recommender
user_id = 1
recommendations = recommend_movies(user_id, user_movie_matrix_norm, item_similarity, item_genre_matrix, weighted_similarity)
print("Recommended movies for user {}: {}".format(user_id, recommendations))


KeyboardInterrupt: 